In [6]:
import sys
from typing import Dict, Tuple, Optional
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

sys.path.insert(0, "../")

from mdu.eval.table_analysis_utils import (
    transform_by_tasks,
    compute_average_ranks,
)

from configs.interesting_compositions import INTERESTING_COMPOSITIONS
from table_utils import (
    load_and_combine_csv_files,
    analyze_hyperparameter_effects_general,
    analyze_hyperparameter_effects_per_dataset,
    compare_hyperparameters_by_metric,
    analyze_individual_component_performance,
)


# Set pandas display options to show all columns
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)


In [7]:
# Load and combine all per_comp_scaled_benchmark CSV files
pattern = "extended_benchmark_*.csv"
base_dir = "../resources/extended_benchmark"
combined_df, loading_summary = load_and_combine_csv_files(pattern=pattern, base_dir=base_dir)

print("Loading Summary:")
display(loading_summary)

print(f"\nCombined DataFrame shape: {combined_df.shape}")

# Create summary of hyperparameters
hyperparam_cols = ['eps', 'grid_size', 'n_targets_multiplier', 'scaler_type', 'target']
hyperparam_summary = []

for col in hyperparam_cols:
    if col in combined_df.columns:
        unique_vals = sorted(combined_df[col].unique())
        hyperparam_summary.append({
            'hyperparameter': col,
            'unique_values': unique_vals,
            'count': len(unique_vals)
        })

hyperparam_df = pd.DataFrame(hyperparam_summary)
print("\nHyperparameter Summary:")
display(hyperparam_df)

# Dataset summary
dataset_summary = pd.DataFrame({
    'dataset': sorted(combined_df['ind_dataset'].unique()),
    'row_count': [len(combined_df[combined_df['ind_dataset'] == ds]) for ds in sorted(combined_df['ind_dataset'].unique())]
})

print("\nDataset Summary:")
display(dataset_summary)

print(f"\nTotal source files loaded: {len(combined_df['source_file'].unique())}")


Loading Summary:


,filename,rows,eps,iters,tol,rs,grid_size,n_targets_multiplier,target,scaler_type,experiment_type
0,extended_benchmark_entropic_target_beta_eps_2.0_iters_150_tol_1e-06_rs_42_grid_size_5_n_targets_multiplier_1.csv,7920,2.0,150,0.000001,150,5,1,beta,none,extended_benchmark
1,extended_benchmark_entropic_target_beta_eps_2.0_iters_150_tol_1e-06_rs_42_grid_size_5_n_targets_multiplier_2.csv,7920,2.0,150,0.000001,150,5,2,beta,none,extended_benchmark
2,extended_benchmark_entropic_target_beta_eps_2.0_iters_150_tol_1e-06_rs_42_grid_size_2_n_targets_multiplier_2.csv,7920,2.0,150,0.000001,150,2,2,beta,none,extended_benchmark
3,extended_benchmark_entropic_target_beta_eps_2.0_iters_150_tol_1e-06_rs_42_grid_size_2_n_targets_multiplier_1.csv,7920,2.0,150,0.000001,150,2,1,beta,none,extended_benchmark
4,extended_benchmark_entropic_target_exp_eps_2.0_iters_150_tol_1e-06_rs_42_grid_size_5_n_targets_multiplier_1.csv,7920,2.0,150,0.000001,150,5,1,exp,none,extended_benchmark
5,extended_benchmark_entropic_target_exp_eps_2.0_iters_150_tol_1e-06_rs_42_grid_size_5_n_targets_multiplier_2.csv,7920,2.0,150,0.000001,150,5,2,exp,none,extended_benchmark
6,extended_benchmark_entropic_target_exp_eps_2.0_iters_150_tol_1e-06_rs_42_grid_size_2_n_targets_multiplier_2.csv,7920,2.0,150,0.000001,150,2,2,exp,none,extended_benchmark
7,extended_benchmark_entropic_target_exp_eps_2.0_iters_150_tol_1e-06_rs_42_grid_size_2_n_targets_multiplier_1.csv,7920,2.0,150,0.000001,150,2,1,exp,none,extended_benchmark
8,extended_benchmark_entropic_target_exp_eps_0.5_iters_150_tol_1e-06_rs_42_grid_size_5_n_targets_multiplier_2_global_scaler.csv,7680,0.5,150,0.000001,150,5,2,exp,global_scaler,extended_benchmark
9,extended_benchmark_entropic_target_beta_eps_2.0_iters_150_tol_1e-06_rs_42_grid_size_2_n_targets_multiplier_1_global_scaler.csv,7680,2.0,150,0.000001,150,2,1,beta,global_scaler,extended_benchmark



Combined DataFrame shape: (249600, 33)

Hyperparameter Summary:


,hyperparameter,unique_values,count
0,eps,"[0.5, 2.0]",2
1,grid_size,"[2, 5]",2
2,n_targets_multiplier,"[1, 2]",2
3,scaler_type,"[global_scaler, none]",2
4,target,"[beta, exp]",2



Dataset Summary:


,dataset,row_count
0,cifar10,83200
1,cifar100,83200
2,tiny_imagenet,83200



Total source files loaded: 32


In [8]:
# Analyze hyperparameter effects across all datasets
print("Analyzing hyperparameter effects across all datasets...")

# Focus on a subset of composite measures for initial analysis
# key_composites = [
#     "COMPOSITE BAYES ALL OUTER",
#     "COMPOSITE BAYES ALL INNER", 
#     "COMPOSITE BAYES ALL CENTRAL",
#     "COMPOSITE EXCESS ALL OUTER OUTER"
# ]

key_composites = list(INTERESTING_COMPOSITIONS.keys())  

general_results, analysis_summary = analyze_hyperparameter_effects_general(
    combined_df, 
    composite_names=key_composites
)

print("\nAnalysis Summary:")
display(analysis_summary)


Analyzing hyperparameter effects across all datasets...
Note: Composite measure 'composite excess all inner inner' not found in DataFrame columns
Note: Composite measure 'composite excess all inner inner' not found in DataFrame columns
Note: Composite measure 'composite excess all inner inner' not found in DataFrame columns
Note: Composite measure 'composite excess all inner inner' not found in DataFrame columns
Note: Composite measure 'composite excess all inner inner' not found in DataFrame columns
Note: Composite measure 'composite excess all inner inner' not found in DataFrame columns
Note: Composite measure 'composite excess all inner inner' not found in DataFrame columns
Note: Composite measure 'composite excess all inner inner' not found in DataFrame columns
Note: Composite measure 'composite excess all inner inner' not found in DataFrame columns
Note: Composite measure 'composite excess all inner inner' not found in DataFrame columns
Note: Composite measure 'composite excess al

,hyperparameter_combination,rows_count,eps,grid_size,n_targets_multiplier,target,scaler_type
0,"eps=0.5, grid_size=2, n_targets_multiplier=1, target=beta, scaler_type=global_scaler",7680,0.5,2,1,beta,global_scaler
1,"eps=0.5, grid_size=2, n_targets_multiplier=1, target=beta, scaler_type=none",7920,0.5,2,1,beta,none
2,"eps=0.5, grid_size=2, n_targets_multiplier=1, target=exp, scaler_type=global_scaler",7680,0.5,2,1,exp,global_scaler
3,"eps=0.5, grid_size=2, n_targets_multiplier=1, target=exp, scaler_type=none",7920,0.5,2,1,exp,none
4,"eps=0.5, grid_size=2, n_targets_multiplier=2, target=beta, scaler_type=global_scaler",7680,0.5,2,2,beta,global_scaler
5,"eps=0.5, grid_size=2, n_targets_multiplier=2, target=beta, scaler_type=none",7920,0.5,2,2,beta,none
6,"eps=0.5, grid_size=2, n_targets_multiplier=2, target=exp, scaler_type=global_scaler",7680,0.5,2,2,exp,global_scaler
7,"eps=0.5, grid_size=2, n_targets_multiplier=2, target=exp, scaler_type=none",7920,0.5,2,2,exp,none
8,"eps=0.5, grid_size=5, n_targets_multiplier=1, target=beta, scaler_type=global_scaler",7680,0.5,5,1,beta,global_scaler
9,"eps=0.5, grid_size=5, n_targets_multiplier=1, target=beta, scaler_type=none",7920,0.5,5,1,beta,none


In [9]:
# Display results for each composite measure
for composite_name, results_df in general_results.items():
    print(f"\n{'='*60}")
    print(f"RESULTS FOR: {composite_name}")
    print(f"{'='*60}")
    display(results_df.sort_values('avg_rank', ascending=True))
    
    if not results_df.empty and 'pareto_percentage' in results_df.columns:
        # Create best hyperparameter summary
        best_pareto_idx = results_df['pareto_percentage'].idxmax()
        best_pareto_row = results_df.loc[best_pareto_idx]
        
        best_dominance_idx = results_df['dominates_50_pct'].idxmax() if 'dominates_50_pct' in results_df.columns else None
        best_dominance_row = results_df.loc[best_dominance_idx] if best_dominance_idx is not None else None
        
        summary_data = []
        
        # Best by Pareto percentage
        summary_data.append({
            'metric': 'Best Pareto %',
            'value': f"{best_pareto_row['pareto_percentage']:.1f}%",
            'eps': best_pareto_row.get('eps', 'N/A'),
            'grid_size': best_pareto_row.get('grid_size', 'N/A'),
            'n_targets_multiplier': best_pareto_row.get('n_targets_multiplier', 'N/A'),
            'scaler_type': best_pareto_row.get('scaler_type', 'N/A'),
            'target': best_pareto_row.get('target', 'N/A')
        })
        
        # Best by dominance
        if best_dominance_row is not None:
            summary_data.append({
                'metric': 'Best Dominance 50%',
                'value': f"{best_dominance_row['dominates_50_pct']}/{best_dominance_row['n_problems']}",
                'eps': best_dominance_row.get('eps', 'N/A'),
                'grid_size': best_dominance_row.get('grid_size', 'N/A'),
                'n_targets_multiplier': best_dominance_row.get('n_targets_multiplier', 'N/A'),
                'scaler_type': best_dominance_row.get('scaler_type', 'N/A'),
                'target': best_dominance_row.get('target', 'N/A')
            })
        
        best_summary_df = pd.DataFrame(summary_data)
        print("\nBest Hyperparameter Combinations:")
        display(best_summary_df)
    
    # Add individual component analysis
    print(f"\n{'-'*40}")
    print(f"INDIVIDUAL COMPONENT ANALYSIS FOR: {composite_name}")
    print(f"{'-'*40}")
    
    try:
        individual_stats = analyze_individual_component_performance(
            combined_df, composite_name, selective_metric="acc_cov_auc"
        )
        
        if not individual_stats.empty:
            print("Individual Component Performance:")
            display(individual_stats[['component', 'avg_rank', 'dominates_50_pct', 'dominates_75_pct', 
                                    'dominates_100_pct', 'dominates_50_pct_rate', 'dominates_75_pct_rate', 
                                    'dominates_100_pct_rate', 'n_problems']])
            
            # Summary of best individual components
            if len(individual_stats) > 0:
                best_component = individual_stats.iloc[0]
                print(f"\nBest Individual Component: {best_component['component']}")
                print(f"  - Average Rank: {best_component['avg_rank']:.2f}")
                print(f"  - Dominates 50% in {best_component['dominates_50_pct']}/{best_component['n_problems']} problems ({best_component['dominates_50_pct_rate']:.1f}%)")
                print(f"  - Dominates 75% in {best_component['dominates_75_pct']}/{best_component['n_problems']} problems ({best_component['dominates_75_pct_rate']:.1f}%)")
                print(f"  - Dominates 100% in {best_component['dominates_100_pct']}/{best_component['n_problems']} problems ({best_component['dominates_100_pct_rate']:.1f}%)")
        else:
            print("No individual component data available.")
            
    except Exception as e:
        print(f"Error analyzing individual components: {e}")
        import traceback
        traceback.print_exc()



RESULTS FOR: COMPOSITE BAYES ALL OUTER


,eps,grid_size,n_targets_multiplier,target,scaler_type,n_problems,dominates_100_pct,dominates_75_pct,dominates_50_pct,beats_worst_component,avg_rank,pareto_count,pareto_total,pareto_percentage
2,0.5,2,1,exp,global_scaler,15,1,9,11,15,28.266667,57,105,54.285714
6,0.5,2,2,exp,global_scaler,15,1,8,11,15,28.266667,49,105,46.666667
10,0.5,5,1,exp,global_scaler,15,1,9,11,15,28.266667,57,105,54.285714
14,0.5,5,2,exp,global_scaler,15,1,8,11,15,28.533333,49,105,46.666667
26,2.0,5,1,exp,global_scaler,15,1,9,11,15,28.600000,57,105,54.285714
18,2.0,2,1,exp,global_scaler,15,1,9,11,15,28.666667,57,105,54.285714
4,0.5,2,2,beta,global_scaler,15,1,8,11,15,28.800000,49,105,46.666667
12,0.5,5,2,beta,global_scaler,15,1,8,11,15,28.800000,49,105,46.666667
22,2.0,2,2,exp,global_scaler,15,1,8,11,15,28.800000,49,105,46.666667
30,2.0,5,2,exp,global_scaler,15,1,8,11,15,28.866667,49,105,46.666667



Best Hyperparameter Combinations:


,metric,value,eps,grid_size,n_targets_multiplier,scaler_type,target
0,Best Pareto %,54.3%,0.5,2,1,global_scaler,exp
1,Best Dominance 50%,11/15,0.5,2,1,global_scaler,beta



----------------------------------------
INDIVIDUAL COMPONENT ANALYSIS FOR: COMPOSITE BAYES ALL OUTER
----------------------------------------
Individual Component Performance:


...


Best Hyperparameter Combinations:


,metric,value,eps,grid_size,n_targets_multiplier,scaler_type,target
0,Best Pareto %,83.8%,2.0,2,1,none,beta
1,Best Dominance 50%,15/15,0.5,2,1,none,beta



----------------------------------------
INDIVIDUAL COMPONENT ANALYSIS FOR: COMPOSITE EAT LOGSCORE OUTER OUTER + M
----------------------------------------
Individual Component Performance:


,component,avg_rank,dominates_50_pct,dominates_75_pct,dominates_100_pct,dominates_50_pct_rate,dominates_75_pct_rate,dominates_100_pct_rate,n_problems
1,R_t 1 1 (Logscore),20.666667,15,13,11,100.000000,86.666667,73.333333,15
2,R_b 1 (Logscore),26.866667,15,12,4,100.000000,80.000000,26.666667,15
0,R_e 1 1 (Logscore),41.200000,13,3,0,86.666667,20.000000,0.000000,15
3,mahalanobis,78.000000,2,2,0,13.333333,13.333333,0.000000,15



Best Individual Component: R_t 1 1 (Logscore)
  - Average Rank: 20.67
  - Dominates 50% in 15/15 problems (100.0%)
  - Dominates 75% in 13/15 problems (86.7%)
  - Dominates 100% in 11/15 problems (73.3%)
